In [ ]:
from tqdm import tqdm, trange
import yfinance as yf
import pandas as pd
import pickle

In [ ]:
# Load D:\financial-data\marketsmith\all\2024-08-16.parquet
df = pd.read_parquet('D:/financial-data/marketsmith/all/2024-08-16.parquet')
df = df[df['Current Price'] > 1]
symbols = df['Symbol'].to_list()
len(symbols)

In [ ]:
def download_data(symbols_):
    # iterate over symbols by 1000
    df_yahoos = []
    for i in trange(0, len(symbols_), 1000):
        df_yahoo = yf.download(symbols_[i:i+1000])
        df_yahoos.append(df_yahoo)
    
    df_yahoos = pd.concat(df_yahoos)
    
    # Extract unique second elements from the column names
    unique_symbols = set(col[1] for col in df_yahoos.columns)
    
    # Create a dictionary to store DataFrames for each unique second element
    grouped_dfs = {}
    
    # Iterate over each unique second element
    for symbol in tqdm(unique_symbols):
        columns_for_symbol = [col for col in df_yahoos.columns if col[1] == symbol]
        df_temp = df_yahoos[columns_for_symbol].copy()
        df_temp.columns = df_temp.columns.droplevel(1)
        df_temp = df_temp[['Close', 'High', 'Low', 'Open', 'Volume']]
        df_temp = df_temp.dropna(how='all')
        grouped_dfs[symbol] = df_temp
        
    return grouped_dfs

In [ ]:
grouped_dfs = download_data(symbols)

In [ ]:
with open('data/grouped_dfs.pkl', 'wb') as f:
    pickle.dump(grouped_dfs, f)